In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from scipy import stats,sparse
from sklearn.base import TransformerMixin
from datetime import datetime as dt
from math import isnan
from numpy import ma
import pickle as pickle
import xgboost as xgb
import time
from pandas import *
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier 

import json
from sklearn.metrics import roc_curve, auc
from re import sub
from collections import defaultdict
import scipy as sp

from sklearn.cross_validation import StratifiedKFold,cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix
from sklearn import linear_model
from sklearn import metrics


In [6]:
FeaturesSelected=np.load('../../../input/SDGLogistic_Oct172015032138_AUC_0.768923541152.p')
X=np.load('../../../input/nxtrain_standard_original0.npy')
X1= np.load('../../../input/nxtrain_standard_derived0.npy')
X2=pd.read_pickle("../../../input/time_series_derived_standard_train2.dat")
X3=pd.read_pickle("../../../input/time_series_original_standard_train2.dat")
X=np.hstack((X,X1,X2,X3))
y=pickle.load(open("../../../input/ytrain2.dat","rb"))
xtrain=X[:,FeaturesSelected]
del X,X1,X2,X3

X_test=np.load('../../../input/nxtest_standard_original0.npy')
X_test1= np.load('../../../input/nxtest_standard_derived0.npy')
X_test2=pd.read_pickle("../../../input/time_series_derived_standard_test2.dat")
X_test3=pd.read_pickle("../../../input/time_series_original_standard_test2.dat")
X_test=np.hstack((X_test,X_test1,X_test2,X_test3))
xtest=X_test[:,FeaturesSelected]
del X_test1,X_test2,X_test3
TestID=pd.read_pickle("../../../input/TestID.p")

## SGD Logistic Regression

In [12]:
n_iterations=500
KFoldNumber=3
kfold = StratifiedKFold(y, KFoldNumber,shuffle=True, random_state=42)
ypreds_test_wl=pd.DataFrame()
ypreds_train_wl=pd.DataFrame()
ypreds_cvindex_wl=pd.DataFrame()
AUCValues=np.zeros(KFoldNumber)

clf = linear_model.SGDClassifier(loss='log',average=True,n_iter=n_iterations)


kfoldnum=0
train_predicted=np.zeros(y.shape)
for train,cv in kfold:
    kfoldnum=kfoldnum+1
    cv_index=np.zeros(y.shape)
    MODEL = clf.fit(xtrain[train,:], y[train])
    preds_cv = MODEL.predict_proba(xtrain[cv,:])[:,1]
    preds_fulltrain = MODEL.predict_proba(xtrain)[:,1]
    preds_fulltest = MODEL.predict_proba(xtest)[:,1]
    cv_index[cv]=1
    AUC1=metrics.roc_auc_score(y[cv], preds_cv)
    ypreds_test_wl['CV'+ str(kfoldnum)] = preds_fulltest
    ypreds_train_wl['CV'+ str(kfoldnum)] = preds_fulltrain
    ypreds_cvindex_wl['CV'+ str(kfoldnum)] = cv_index
    AUCValues[kfoldnum-1]=float(AUC1)
    print(AUCValues)

trainPredict=(ypreds_train_wl*ypreds_cvindex_wl).sum(axis=1)
testPredictMean=ypreds_test_wl.mean(axis=1)
AUC1=metrics.roc_auc_score(y, trainPredict)
print(AUC1)

str1=time.strftime("%b%d%Y%H%M%S", time.localtime())
str2= ' _AUC_0p'+ str(int(AUC1.mean()*1e6))
filename0='FinalModelPrediction_Logistic_YF'+str1+'_niter' + str(n_iterations)+str2
filename=filename0 +'.p'

pickle.dump((ypreds_train_wl,ypreds_test_wl,ypreds_cvindex_wl,AUCValues), 
            open(filename,'wb'), protocol =2) 

save_train_name=filename0 +'_wl_ypredtrain.pkl'
save_test_name=filename0 +'_wl_ypredtest.pkl'

pickle.dump(trainPredict,open(save_train_name,'wb'),protocol=2)
pickle.dump(testPredictMean,open(save_test_name,'wb'),protocol=2)

print('Model prediction done!')

[ 0.76617212  0.          0.        ]
[ 0.76617212  0.77236154  0.        ]
[ 0.76617212  0.77236154  0.77012077]
0.76953169879
Model prediction done!


## SDG SVM

In [7]:
KFoldNumber=3
niter=400
    
kfold = StratifiedKFold(y, KFoldNumber,shuffle=True, random_state=42)
ypreds_train=[]
ypreds_test_wl=pd.DataFrame()
ypreds_train_wl=pd.DataFrame()
ypreds_cvindex_wl=pd.DataFrame()
indexes_cv=[]
cv_values1=np.zeros(KFoldNumber)

i=0
model = linear_model.SGDClassifier()
kfoldnum=0

train_predicted=np.zeros(y.shape)

for train,cv in kfold:
    kfoldnum=kfoldnum+1
    preds_cv=0
    preds_test=0
    preds_train=0
    ncount=0
    cv_index=np.zeros(y.shape)
    for i in range(niter):
        clf = model.fit(xtrain[train,:], y[train])
        predcv0=clf.predict(xtrain[cv,:])
        predtest0=clf.predict(xtest)
        predtrain0=clf.predict(xtrain)
        preds_cv=preds_cv+predcv0
        preds_test=preds_test+predtest0
        preds_train=preds_train+predtrain0
        ncount+=1
        if np.mod(i,round(niter/10))==0:print(i,end=' ')
    print(' ')
    preds_cv=preds_cv/float(ncount)
    preds_test=preds_test/float(ncount)
    preds_train=preds_train/float(ncount)
    cv_index[cv]=1
    AUC1=metrics.roc_auc_score(y[cv], preds_cv) 
    ypreds_test_wl['CV'+ str(kfoldnum)] = preds_test
    ypreds_train_wl['CV'+ str(kfoldnum)] = preds_train
    ypreds_cvindex_wl['CV'+ str(kfoldnum)] = cv_index
    cv_values1[kfoldnum-1]=float(AUC1)
    print(cv_values1)
    i+=1
    
    
str1=time.strftime("%b%d%Y%H%M%S", time.localtime())
filename='FinalModelPrediction_SVM_'  +  str1 + '_niter400_AUC'+ str(cv_values1.max())+ '.p'
pickle.dump((ypreds_test_wl,ypreds_train_wl,ypreds_cvindex_wl,cv_values1), 
            open(filename,'wb'), protocol =2)  
print('Model prediction done!')

0 40 80 120 160 200 240 280 320 360  
[ 0.76465555  0.          0.        ]
0 40 80 120 160 200 240 280 320 360  
[ 0.76465555  0.77017507  0.        ]
0 40 80 120 160 200 240 280 320 360  
[ 0.76465555  0.77017507  0.76904979]
Model prediction done!


## SGD Passive Agreessive

In [ ]:
KFoldNumber=3
niter=200
    
kfold = StratifiedKFold(y, KFoldNumber,shuffle=True, random_state=42)
ypreds_train=[]
ypreds_test_wl=pd.DataFrame()
ypreds_train_wl=pd.DataFrame()
ypreds_cvindex_wl=pd.DataFrame()
indexes_cv=[]
cv_values1=np.zeros(KFoldNumber)

i=0

model= linear_model.PassiveAggressiveClassifier(loss='squared_hinge',C=1.0)
kfoldnum=0

train_predicted=np.zeros(y.shape)

for train,cv in kfold:
    kfoldnum=kfoldnum+1
    preds_cv=0
    preds_test=0
    preds_train=0
    ncount=0
    cv_index=np.zeros(y.shape)
    for i in range(niter):
        clf = model.fit(xtrain[train,:], y[train])
        predcv0=clf.predict(xtrain[cv,:])
        predtest0=clf.predict(xtest)
        predtrain0=clf.predict(xtrain)
        preds_cv=preds_cv+predcv0
        preds_test=preds_test+predtest0
        preds_train=preds_train+predtrain0
        ncount+=1
        if np.mod(i,round(niter/10))==0:print(i,end=' ')
    print(' ')
    preds_cv=preds_cv/float(ncount)
    preds_test=preds_test/float(ncount)
    preds_train=preds_train/float(ncount)
    cv_index[cv]=1
    AUC1=metrics.roc_auc_score(y[cv], preds_cv) 
    ypreds_test_wl['CV'+ str(kfoldnum)] = preds_test
    ypreds_train_wl['CV'+ str(kfoldnum)] = preds_train
    ypreds_cvindex_wl['CV'+ str(kfoldnum)] = cv_index
    cv_values1[kfoldnum-1]=float(AUC1)
    print(cv_values1)
    i+=1
    
    
str1=time.strftime("%b%d%Y%H%M%S", time.localtime())
filename='FinalModelPrediction_PassiveAggressive_'+str1+'_niter800_AUC'+ str(cv_values1.max())+ '.p'
pickle.dump((ypreds_test_wl,ypreds_train_wl,ypreds_cvindex_wl,cv_values1), 
            open(filename,'wb'), protocol =2)  
print('Model prediction done!')